In [1]:
#define all relevant functions
import numpy as np
import pandas as pd
from scipy.fft import fftn, ifftn, fftshift
import re

import matplotlib.pyplot as plt

from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches


In [4]:
def read_data(file_name):
    """Reads and processes data from a given file, correctly handling filenames with 'ffX'."""
    data = []
    
    with open(file_name, 'r') as f:
        for line in f:
            l = line.split()
            
            if len(l) != 4:
                continue  # Skip lines that do not match expected format
            
            try:
                # Extract the filename and split into parts
                file_parts = l[0].split("/")[-1].split("_")

                # Remove any "ffX" component
                file_parts = [part for part in file_parts if not (part.startswith("ff") and part[2:].isdigit())]

                # Extract only the LAST 4 elements (since the earlier ones are not needed)
                U = float(file_parts[-4])   # Extracts U parameter
                J = float(file_parts[-3])   # Extracts J parameter
                nk = int(file_parts[-2])    # Extracts nk number points
                nkf = int(file_parts[-1].split(".")[0])  # Extracts nkf number points

                Tc = l[1]  # Tc value
                phase = float(l[2])  # Dominant phase
                
                # Extract time in seconds
                time_parts = l[3].split("m")
                seconds = float(time_parts[0]) * 60 + float(time_parts[1].split("s")[0])
                
                # Store data as a LIST (not dictionary)
                final_data = [U, J, nk, nkf, Tc, phase, seconds]
                data.append(final_data)
            
            except (ValueError, IndexError) as e:
                print(f"Skipping line due to error: {line.strip()} - {e}")
    
    return data


In [5]:
def plot_Tc(data, U):

    #specify U for which th Tc is plotted 
    data = read_data(data)

    groundstate = []
    Us = []
    mu = []
    points = []
    tc = []
    nk =[]
    nkf = []
    times = []
    for d in data:
        if d[0] == U:

            Us.append(d[0])
            mu.append(d[1])
            points.append([d[2],d[3]])  #nk, nkf vector
            groundstate.append(d[4])  #Stores GS
            nk.append(d[2])
            nkf.append(d[3])
            tc.append(d[5])  #stores Tc
            times.append(d[6])
    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.grid()

    color_map = {
        "SC": "#A7C7E7",
        "CDW": "#FFB347",
        "FL": "black",
        "SDW": "#B0E57C"
    }

    labels_added = {}

    for i in range(len(Us)):
        state = groundstate[i]
        colour = color_map.get(state, "white")

    # Add label only if it hasn't been added before
        label = state if state not in labels_added else None
        labels_added[state] = True  # Mark label as used
        ax.scatter(mu[i], tc[i], c=colour, s=120, marker="o", label=label)
    
    ax.plot(mu, tc, ls ='--', color = 'grey')
   # ax.axvline(-3.85, ls ='--', color = 'black')
    #ax.axvline(-2.15, ls ='--', color = 'black')
    #ax.axvline(-1.85, ls ='--', color = 'black')
    #ax.axvline(3.85, ls ='--', color = 'black')
    #ax.axvline(2.15, ls ='--', color = 'black')
    #ax.axvline(1.85, ls ='--', color = 'black')
    #ax.axvline(-1.35, ls ='--', color = 'black')
    #ax.axvline(1.35, ls ='--', color = 'black')
    #ax.axvline(-0.65, ls ='--', color = 'black')
    #ax.axvline(0.65, ls ='--', color = 'black')
    ax.tick_params('both', labelsize = 20)
    ax.legend(fontsize =20)
    ax.set_xlabel(r"$\mu$(eV)", fontsize = 20)
    ax.set_ylabel("Tc(eV)", fontsize =20)